In [ ]:
# Plot time series of precipitation
# Only 2 models (50 member ENS and Neighbourhood from DestinE Extremes DT)
# Obs from STVL
# The closest synop station is found for the input location, model data is plotted for the station location
# Automatic MARS retrieval is needed

In [ ]:
%env MIR_GRIB_INPUT_BUFFER_SIZE=7688961960
%env MARS_READANY_BUFFER_SIZE=7688961960
%env MIR_CACHE_PATH=${SCRATCH}

In [ ]:
import metview as mv
import numpy as np
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import pandas as pd
import numpy as np
import os
import csv
import math
from matplotlib.patches import Patch  # Import Patch to create custom legend handles
import vtb

In [ ]:
class CustomDateFormatter(mdates.DateFormatter):
    def __init__(self):
        super().__init__()

    def __call__(self, x, pos=0):
        if x.hour == 0 and x.minute == 0:
            return x.strftime('%Y-%m-%d %H UTC')
        else:
            return x.strftime('%H UTC')

In [ ]:
adate = datetime(2025,5,2)                  # Forecast initialisation day
atime = 0                                    # Forecast initialisation time

# the ensemble size (perturbed members)
ens_num_oper = 50

# location_in = [47.4979, 19.0402]   # Budapest, Hungary
# location_name = "Budapest, Hungary"

# location_in = [45.4384, 10.9917]  
# location_name = "Verona, Italy"

# location_in = [50.7374, 7.0982]  
# location_name = "Bonn, Germany"   # North of Konigswinter

location_in = [50.85949, 7.13899]  
location_name = "CGN Airport, Germany"

path_in_EDT = "/ec/vol/destine/neighbourhood/edt/"
path_in_ENS = "/ec/vol/destine/neighbourhood/oper_ENS_9km/"
path_in_NB = "/ec/vol/destine/neighbourhood/percentile/"

In [ ]:
# Get the nearest synop to the requested location

end_date = adate + timedelta(hours=120)

my_dates = pd.date_range(adate,end_date,freq="6h")

# r = mv.stvl(
r = vtb.media.stvl_retrieve( 
    sources="synop",
    parameter="tp",
    date=my_dates,
    #times=6,
    period=pd.Timedelta(hours=6)
)


In [ ]:
r3 = r.to_metview()[0]
dist = mv.distance(r3, location_in)
dd = dist.values()
loc_closest_index = np.nanargmin(dd)
rr = r.to_dataframe()
location = [rr.latitude[loc_closest_index], rr.longitude[loc_closest_index]]  # location of closest synop (model values will be retrived for this location)
obs_loc = rr.values[loc_closest_index][4:]     # vector of observation values

In [ ]:
location

In [ ]:
dist[loc_closest_index]

In [ ]:
path_file_in1cf = adate.strftime("%Y%m%d")  + "_cf.grib" 
path_file_in1pf = adate.strftime("%Y%m%d")  + "_pf.grib" 
path_file_in1edt = adate.strftime("%Y%m%d")  + ".grib" 

print(path_file_in1cf)

In [ ]:
# download oper ENS data from MARS if not downloaded already
if os.path.isfile(path_in_ENS + path_file_in1pf):
    print("ENS file already exists, no MARS retrieval. : ", path_in_ENS + path_file_in1pf)
else:
    print("Retriving ENS data from  MARS")
    ret_field = mv.retrieve(
        date= adate,
        time= 0,
        step= [0,"to",120,"by",6],
        type= "pf",
        number=[1,"to",50],
        class_= "od",
        stream= "enfo",
        levtype= "sfc",
        expver= 1,
        param="228.128"     # total precipitation
        
    )

    # save to file
    print("writing retrieved file: ", path_in_ENS + path_file_in1pf)
    mv.write(path_in_ENS + path_file_in1pf,ret_field)

    ret_field = mv.retrieve(
        date= adate,
        time= 0,
        step= [0,"to",120,"by",6],
        type= "cf",
        class_= "od",
        stream= "enfo",
        levtype= "sfc",
        expver= 1,
        param="228.128"     # total precipitation
        
    )

    # save to file
    print("writing retrieved file: ", path_in_ENS + path_file_in1cf)
    mv.write(path_in_ENS + path_file_in1cf,ret_field)


In [ ]:
file_grib1cf = mv.read(path_in_ENS + path_file_in1cf)

file_grib1pf = mv.read(path_in_ENS + path_file_in1pf)

file_grib1 = mv.merge(file_grib1cf, file_grib1pf)

file_grib1edt = mv.read(path_in_EDT + path_file_in1edt)

In [ ]:
tp1cf = file_grib1cf.select(
     shortName="tp",
     )

tp1edt = file_grib1edt.select(
     shortName="tp",
     )

In [ ]:
tp1 = file_grib1.select(
     shortName="tp",
     )

In [ ]:
# extract time series for the pertubations (as a 2D ndarray)
tp1_pert_loc = []
for i in range(1, ens_num_oper + 1):
    tp1_pert = tp1.select(number=i, type="pf")
    tp1_pert_loc.append(mv.nearest_gridpoint(tp1_pert, location))
tp1_pert_loc_arr = np.array(tp1_pert_loc)

# extract time series for the control forecast (as 1D array)
tp1_ctr = tp1.select(type="cf")
tp1_ctr_loc_arr = np.array(mv.nearest_gridpoint(tp1_ctr, location))

# extract time series for EDT forecast (as 1D array)
tp1_edt_loc_arr = np.array(mv.nearest_gridpoint(tp1edt, location))

del tp1_pert_loc,tp1,file_grib1

tp1_pert_loc_arr=tp1_pert_loc_arr*1000
tp1_ctr_loc_arr=tp1_ctr_loc_arr*1000
tp1_edt_loc_arr=tp1_edt_loc_arr*1000

In [ ]:
print(tp1_pert_loc_arr.shape)

In [ ]:
# compute ENS mean series
tp1_mean = np.vstack([tp1_pert_loc_arr, tp1_ctr_loc_arr]).mean(axis=0)

In [ ]:
# get metadata for the title
meta = mv.grib_get(tp1_pert[0], ["name", "level", "date", "time"])[0]

# get the valid times for the time series points
d_times = mv.valid_date(tp1_pert)

# determine number of timesteps
ts_num = len(tp1_pert)

In [ ]:
print(d_times)

In [ ]:
print(tp1_pert_loc_arr[0:4])

In [ ]:
# Calculate 24-hour accumulation
def calculate_24h_accumulation(data):
    return data[:, 4:] - data[:, :-4]  # Take the difference every 4 steps (24-hour accumulation)

# Calculate 6-hour accumulation
def calculate_6h_accumulation(data):
    return data[:, 1:] - data[:, :-1]  # Take the difference between consecutive time steps

# Calculate 6-hour accumulation
def calculate_6h_accumulation_ctr(data):
    return data[1:] - data[:-1]  # Take the difference between consecutive time steps

# Calculate accumulation for each system
tp1_6h_accum = calculate_6h_accumulation(tp1_pert_loc_arr)
tp1_ctr_6h_accum = calculate_6h_accumulation_ctr(tp1_ctr_loc_arr)
tp1_edt_6h_accum = calculate_6h_accumulation_ctr(tp1_edt_loc_arr)

# Use the original time array for 24-hour intervals (no shift for end-of-period accumulation)
d_times_24h = np.array(d_times[4:])  # Keep times aligned with the end of the accumulation period
d_times = np.array(d_times)  # Keep times aligned with the end of the accumulation period

# Function to calculate percentiles and mean
def calculate_percentiles(data):
    p1 = np.percentile(data, 1, axis=0)
    p25 = np.percentile(data, 25, axis=0)
    p75 = np.percentile(data, 75, axis=0)
    p99 = np.percentile(data, 99, axis=0)
    p50 = np.percentile(data, 50, axis=0)
    mean = np.mean(data, axis=0)
    return p1, p25, p75, p99, p50, mean

# Select only times that are at midnight (0 hours) for the end of the accumulation period, or every 6h if we have 6-h accumulations
# midnight_indices = np.where(d_times_24h.astype('datetime64[m]').astype(int) % 1440 == 0)[0]  # Check for midnight (0 minutes)
# d_times_midnight = d_times_24h[midnight_indices]
six_hour_indices = np.where(d_times.astype('datetime64[m]').astype(int) % 360 == 0)[0]  # Check for every 6 hours (360 minutes)
d_times_six_hour = d_times[six_hour_indices]

# Calculate percentiles and mean for each system
tp1_p1, tp1_p25, tp1_p75, tp1_p99, tp1_p50, tp1_mean = calculate_percentiles(tp1_6h_accum)
# tp2_p1, tp2_p25, tp2_p75, tp2_p99, tp2_p50, tp2_mean = calculate_percentiles(tp2_6h_accum)
# tp3_p1, tp3_p25, tp3_p75, tp3_p99, tp3_p50, tp3_mean = calculate_percentiles(tp3_6h_accum)

valid_six_hour_indices = [i for i in six_hour_indices if i < len(tp1_p1)]
print(valid_six_hour_indices)

# Filter the percentiles and mean for plotting at midnight only
tp1_midnight_p1 = tp1_p1[valid_six_hour_indices]
tp1_midnight_p25 = tp1_p25[valid_six_hour_indices]
tp1_midnight_p75 = tp1_p75[valid_six_hour_indices]
tp1_midnight_p99 = tp1_p99[valid_six_hour_indices]
tp1_midnight_p50 = tp1_p50[valid_six_hour_indices]
tp1_midnight_mean = tp1_mean[valid_six_hour_indices]

tp1_midnight_ctr = tp1_ctr_6h_accum[valid_six_hour_indices]
tp1_midnight_edt = tp1_edt_6h_accum[valid_six_hour_indices]


## Extract info neighbourhood technique

In [ ]:
date=adate.strftime("%Y%m%d") 
step_start=6           # first lead time
step_end=120            # last lead time
prec_acc=6              # precip accumulation period

my_point = location

my_perc1=1    # percentile value 
my_perc2=25    # percentile value 
my_perc3=75    # percentile value 
my_perc4=99    # percentile value 
my_perc5=50    # percentile value 

# These should not be changed
NB_radius="VR"         # neighbourhood search radius in km
NB_time=0              # neighbourhood search in time (+-prec_acc)

# read grib data 
file_in1 = path_in_NB + "NB_tp6h_p" + str(float(my_perc1)) + "_r" + str(NB_radius) + "_ST" + str(NB_time) + "_" + date + "_step_" + str(step_start) + "-" + str(step_end) + ".grib2"
f1=mv.read(file_in1)
file_in2 = path_in_NB + "NB_tp6h_p" + str(float(my_perc2)) + "_r" + str(NB_radius) + "_ST" + str(NB_time) + "_" + date + "_step_" + str(step_start) + "-" + str(step_end) + ".grib2"
f2=mv.read(file_in2)
file_in3 = path_in_NB + "NB_tp6h_p" + str(float(my_perc3)) + "_r" + str(NB_radius) + "_ST" + str(NB_time) + "_" + date + "_step_" + str(step_start) + "-" + str(step_end) + ".grib2"
f3=mv.read(file_in3)
file_in4 = path_in_NB + "NB_tp6h_p" + str(float(my_perc4)) + "_r" + str(NB_radius) + "_ST" + str(NB_time) + "_" + date + "_step_" + str(step_start) + "-" + str(step_end) + ".grib2"
f4=mv.read(file_in4)
file_in5 = path_in_NB + "NB_tp6h_p" + str(float(my_perc5)) + "_r" + str(NB_radius) + "_ST" + str(NB_time) + "_" + date + "_step_" + str(step_start) + "-" + str(step_end) + ".grib2"
f5=mv.read(file_in5)

perc_lst1 = []   # list containing percentile values for all lead times
perc_lst2 = []   # list containing percentile values for all lead times
perc_lst3 = []   # list containing percentile values for all lead times
perc_lst4 = []   # list containing percentile values for all lead times
perc_lst5 = []   # list containing percentile values for all lead times

# Loop over all lead times
for my_step in range(step_start, step_end+1,prec_acc):
    perc_NB1 = f1.select(
        shortName="tp",
        stepRange=str(my_step-prec_acc)+"-"+str(my_step),
        )
    perc_NB_point1 = mv.nearest_gridpoint(perc_NB1, my_point)
    perc_lst1.append(perc_NB_point1)

    perc_NB2 = f2.select(
        shortName="tp",
        stepRange=str(my_step-prec_acc)+"-"+str(my_step),
        )
    perc_NB_point2 = mv.nearest_gridpoint(perc_NB2, my_point)
    perc_lst2.append(perc_NB_point2)

    perc_NB3 = f3.select(
        shortName="tp",
        stepRange=str(my_step-prec_acc)+"-"+str(my_step),
        )
    perc_NB_point3 = mv.nearest_gridpoint(perc_NB3, my_point)
    perc_lst3.append(perc_NB_point3)

    perc_NB4 = f4.select(
        shortName="tp",
        stepRange=str(my_step-prec_acc)+"-"+str(my_step),
        )
    perc_NB_point4 = mv.nearest_gridpoint(perc_NB4, my_point)
    perc_lst4.append(perc_NB_point4)

    perc_NB5 = f5.select(
        shortName="tp",
        stepRange=str(my_step-prec_acc)+"-"+str(my_step),
        )
    perc_NB_point5 = mv.nearest_gridpoint(perc_NB5, my_point)
    perc_lst5.append(perc_NB_point5)
    
# Convert date to a datetime object
initial_date = datetime.strptime(date, "%Y%m%d")

# Generate the datetime list
times_nearest = [initial_date + timedelta(hours=step) for step in range(step_start, step_end + prec_acc, prec_acc)]
# Print the resulting times for debugging
print(times_nearest) 
print(perc_lst1) 
print(perc_lst2) 
print(perc_lst3) 
print(perc_lst4) 
print(perc_lst5) 


In [ ]:
# Create a plot
fig, ax = plt.subplots(figsize=(10, 6))

# Define more opaque colors for each system
color1 = 'deepskyblue'  # A more distinct shade of blue for system 1 (ENS 48r1)
#color1='blue'
color2 = 'salmon'       # A more distinct shade of red for system 2 (49r1 ENS)
color3 = 'mediumseagreen'  # A more distinct shade of green for system 3 (DestinE 10 ens)

# Plot shaded areas for tp1_pert_loc_arr (ENS 50)
ax.fill_between(d_times_six_hour[1:], tp1_midnight_p1, tp1_midnight_p99, color=color1, alpha=0.4)  # Increased opacity
ax.fill_between(d_times_six_hour[1:], tp1_midnight_p25, tp1_midnight_p75, color=color1, alpha=0.6)  # Increased opacity
ax.plot(d_times_six_hour[1:], tp1_midnight_p50, color='blue', lw=2.5, label='9km ENS', zorder=10)
ax.plot(d_times_six_hour[1:], tp1_midnight_ctr, color='blue', lw=2.5, label='9km control', zorder=10, linestyle = 'dashed')   # control forecast

# Plot shaded areas for Neighbourhood
ax.fill_between(times_nearest, perc_lst1, perc_lst4, color=color2, alpha=0.4)  # Increased opacity
ax.fill_between(times_nearest, perc_lst2, perc_lst3, color=color2, alpha=0.6)  # Increased opacity
ax.plot(times_nearest, perc_lst5, color='red', lw=2.5, label='Neighbourhood', zorder=10)
ax.plot(times_nearest, tp1_midnight_edt, color='red', lw=2.5, label='Extremes-DT', zorder=10, linestyle = 'dashed')

# Plot the observation data as black dots at the corresponding midnight times
# included NaN for obs if these are not yet available
obs_loc = list(obs_loc)
if len(my_dates) > len(obs_loc):
    for i in range(len(my_dates) - len(obs_loc)):
        obs_loc.append(np.nan)
        
ax.plot(my_dates, obs_loc, 'ko', markersize=8, label='Observations', zorder=15)  # Black dots for observations

# Set the x-axis limit up to 2024-09-17
#end_date = np.datetime64('2025-04-08')
end_date = adate + timedelta(hours=120)
ax.set_xlim([d_times_six_hour[0:][0], end_date])

# Adding labels, title, and grid
ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('6-Hour Accumulated Precipitation', fontsize=12)
ax.set_title('6-Hour Accumulated Precipitation, '+location_name+', lat/lon: '+str(location[0])+' / '+str(location[1]), fontsize=14)
ax.grid(True)

# Format the x-axis to show the dates better
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
fig.autofmt_xdate()

# Adding a simplified legend
handles, labels = ax.get_legend_handles_labels()
percentile_patch = plt.Line2D([0], [0], color='grey', lw=1.5, label='Percentile ranges (1-99, 25-75)')
mean_patch = plt.Line2D([0], [0], color='black', lw=2.5, label='Ensemble median')

# Custom legend with example for percentiles and mean
ax.legend(handles=[percentile_patch] + handles, 
          labels=['Percentile ranges (1-99, 25-75)',
                  'oper ENS 50 member, median', 'oper control','Neighbourhood method from Extremes-DT, median', 'Extremes-DT', 'Observations'],
          loc='upper right', fontsize=10)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
obs_loc